In [10]:
import sys
sys.path.append('../')
from setting import config_read

In [11]:
import pandas as pd
import owlready2
from elasticsearch import Elasticsearch
from mapping import *
from graph import *
from konlpy.tag import Okt

In [12]:
config = config_read('../')

# Load owl file
data_path = config['owl']['path']
onto = owlready2.get_ontology(data_path).load()

# Elasticsearch
server_ip = config['elasticsearch']['ip']
index_name = config['elasticsearch']['name']
es = Elasticsearch(server_ip)

# Load unit path data
unit_path = pd.read_csv('../unit_path.csv')
G = base_graph_generation(unit_path)

# 1. Preprocessing

# 2-1. Query Partitioning

In [13]:
okt = Okt()

# This Love 부른 음악가의 소속 음악 그룹은?
# 용감한 형제가 작곡한 노래의 장르는?
query = input("Question: ")
query_pos = okt.pos(query, norm=True)

query_terms = [q[0] for q in query_pos if q[1] not in ["Josa", "Punctuation"]]

query_patitions = query_patitioning(query_terms)
query_patitions

[['용감한 형제 작곡 노래 장르'],
 ['용감한', '형제 작곡 노래 장르'],
 ['용감한 형제', '작곡 노래 장르'],
 ['용감한 형제 작곡', '노래 장르'],
 ['용감한 형제 작곡 노래', '장르'],
 ['용감한', '형제', '작곡 노래 장르'],
 ['용감한', '형제 작곡', '노래 장르'],
 ['용감한', '형제 작곡 노래', '장르'],
 ['용감한 형제', '작곡', '노래 장르'],
 ['용감한 형제', '작곡 노래', '장르'],
 ['용감한 형제 작곡', '노래', '장르'],
 ['용감한', '형제', '작곡', '노래 장르'],
 ['용감한', '형제', '작곡 노래', '장르'],
 ['용감한', '형제 작곡', '노래', '장르'],
 ['용감한 형제', '작곡', '노래', '장르'],
 ['용감한', '형제', '작곡', '노래', '장르']]

# 2-2. Partition Scoring

In [14]:
p_star, score_list = partition_scoring(es, index_name, query_patitions)
p_star

['용감한 형제', '작곡', '노래', '장르']

# 2-3. Generation of resource combination

In [15]:
candidates_list = [onto.search(label=query) for query in p_star]
resource_combinations = list(itertools.product(*candidates_list))
resource_combinations

[(SKMO_v2.4_0223.용감한_형제,
  SKMO_v2.4_0223.isComposedBy,
  mo.Track,
  SKMO_v2.4_0223.hasGenre),
 (SKMO_v2.4_0223.용감한_형제,
  SKMO_v2.4_0223.isComposedBy,
  mo.Track,
  SKMO_v2.4_0223.Genre)]

# 3. Conceptual graph generation

In [16]:
conceptual_graph = conceptual_graph_generation(onto, resource_combinations)
conceptual_graph

[[(SKMO_v2.4_0223.용감한_형제, SKMO_v2.4_0223.isComposedBy, mo.Track),
  (mo.Track, SKMO_v2.4_0223.hasGenre, None)],
 [(SKMO_v2.4_0223.용감한_형제, SKMO_v2.4_0223.isComposedBy, mo.Track),
  (mo.Track, None, SKMO_v2.4_0223.Genre)]]

# 4. Query graph generation

In [20]:
query_graph = query_graph_generation(G, conceptual_graph)

for k,v in query_graph.items():
    if v == []:
        print(f'{k}의 shortest path')
        print(': 일치하는 shortest path가 없습니다.')
        print() 
    else:
        print(f'{k}의 shortest path')
        for elem in v:
            print(f': {elem}')
        print()

(SKMO_v2.4_0223.용감한_형제, SKMO_v2.4_0223.isComposedBy, mo.Track)의 shortest path
: 일치하는 shortest path가 없습니다.

(mo.Track, SKMO_v2.4_0223.hasGenre, None)의 shortest path
: [('mo.Track', 'SKMO_v2.4_0223.hasGenre', 'SKMO_v2.4_0223.Genre')]

(mo.Track, None, SKMO_v2.4_0223.Genre)의 shortest path
: [('mo.Track', 'SKMO_v2.4_0223.hasGenre', 'SKMO_v2.4_0223.Genre')]

